<a href="https://www.kaggle.com/code/cherant1976gmailcom/chernikov-eda-project-3-model?scriptVersionId=145004888" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Содержание:**
1. [Объявление библиотек](#1)
2. [Загрузка данных](#2)
    + [Загрузка основных данных](#3)
    + [Загрузка вспомогательных данных](#4)
3. [Анализ и первоначальная обработка данных](#5)
4. [Обработка данных для создания признаков](#6)
    + [Сеть отелей](#7)
    + [Слова в отзывах](#8)
    + [Слова в Тегах](#9)
    + [Координаты отеля](#10)
    + [Координаты гостя отеля](#11)
    + [Прочие преобразования признаков](#12)
5. [Создание признаков данных](#13)
    + [Сеть отелей](#14)
    + [Слова в отзывах](#15)
    + [Слова в Тегах](#16)
    + [Создание прочих признаков](#17)
6. [Проверка созданных признаков](#18)
7. [Обучение модели](#19)
8. [Проверка и выгрузка данных](#20)

<a id="1"></a> <br>
# 1. Объявление библиотек

In [ ]:
import pandas as pd
import numpy as np

import requests

import seaborn as sns

import re # библиотека регулярных выражений

from geopy.geocoders import Nominatim
import time

from geopy.extra.rate_limiter import RateLimiter

from geopy import distance

import os

<a id="2"></a> <br>
# 2. Загрузка данных

<a id="3"></a> <br>
## Загрузка основных данных

In [ ]:
hotels_train = pd.read_csv('/kaggle/input/sf-booking/hotels_train.csv')
hotels_test = pd.read_csv('/kaggle/input/sf-booking/hotels_test.csv')
hotels_submission = pd.read_csv('/kaggle/input/sf-booking/submission.csv')

<a id="4"></a> <br>
## Загрузка вспомогательных данных

In [ ]:
# Данные потребуются для задания широты и долготы отеля (в случае отсутствия данных)
# и для определения расстояния между страной, где проживает посетитель отеля и отелем

geolocator = Nominatim(user_agent="http")

url = "http://techslides.com/demos/country-capitals.csv"
req = requests.get(url).content
country_data = pd.read_csv(url,on_bad_lines='skip')
display(country_data.head(3))

<a id="5"></a> <br>
# 3. Анализ и первоначальная обработка данных

In [ ]:
# Простановка признаков отличающих ланные для обучения моделии
# от данных ддля итоговой проверки модели (submission)
hotels_train['train'] = 1
hotels_test['train'] = 0
hotels_test['reviewer_score'] = 0 # простановка отсутствующего признака для данных submission

dupl_columns = list(hotels_train.columns)
#dupl_columns.remove('id')

print(f'Размер: {hotels_test.shape[0]}')

# Удаляем дубликаты:
hotels_train= hotels_train.drop_duplicates()
print('Размер таблицы после удаления дубликатов: {}'.format(hotels_train.shape))

# Объединяем данные
hotels = pd.concat([hotels_train,hotels_test], ignore_index = True)

is_train = hotels['train'] == 1 # Задание маски для данных с заполненным признаком **reviewer_score**

display(hotels.head(3))
display(hotels.info())

Из вывода обычной общей информация о данных видно, что для ряда строк данных отсутсвует информация по широте **lat** и долготе **lng**

Анализ наличия выбросов по оценке отзыва **reviewer_score**

In [ ]:
sns.boxplot(data=hotels[is_train], x='reviewer_score');

Сразу напишу результаты проверки по выбросам (сделана ниже). Такие значения имеют право быть.

In [ ]:
mask = ( hotels['reviewer_score'] < 3 ) & ( is_train )
display(hotels[mask].head(3))

<a id="6"></a> <br>
# 4. Подготовка данных для создания признаков

<a id="7"></a> <br>
## Сеть отелей

Определение по ключевому слову **by** сетей отелей, присутствующих в данных.

In [ ]:
# С использованием слова by и регулярных выражений нахожу названия сетей отелей
hotels['hotel_name'] = hotels['hotel_name'].str.lower()
mask = hotels['hotel_name'].str.contains(' by ')
display( hotels[mask]['hotel_name'].str.extract(r'((?<=\sby\s)[a-z]*)', expand=False).unique() )

Среди сетей отелей есть **mh** и **hc**. Так как могут быть слова начинающиеся или заканчивыающиеся на **mh** и/или на **hc**, то решил выделить эти названия, предполагая, что либо до них либо сразу после них должен быть пробел. Сделал проверку этого предположения.

In [ ]:
mask = hotels['hotel_name'].str.contains(' hc')
display( hotels[mask]['hotel_name'].unique() )
mask = hotels['hotel_name'].str.contains('hc ')
display( hotels[mask]['hotel_name'].unique() )
mask = hotels['hotel_name'].str.contains(' mh')
display( hotels[mask]['hotel_name'].unique() )
mask = hotels['hotel_name'].str.contains('mh ')
display( hotels[mask]['hotel_name'].unique() )

Решил "вручную" ввести найденные названия сетей.

In [ ]:
hotel_chain_list = ['hilton','marriott','hyatt','elegancia',' mh','mh ','sofitel', \
    'happyculture','axel','radisson','fraser',' hc','hc ','jumeirah','melia','montcalm']

Признак сети отелей

In [ ]:
# Создание признака сети, к которой относится отель
# Признак равен или названию сети, если сеть есть в названии отеля
# иначе признак просто равен названию отеля

def f_hotel_chain(x):
    str_hotel_name = x['hotel_name'].lower()
    for chain in hotel_chain_list:
        if chain in str_hotel_name:
            return chain.strip() # для порядка strip )
    return str_hotel_name
         
hotels['hotel_chain'] =  hotels.apply( lambda x : f_hotel_chain(x), axis=1 )

display(hotels.tail(3))

<a id="8"></a> <br>
## Слова в отзывах

### Анализ слов в положительных отзывах

In [ ]:
list_tag=[]
dict_tag = dict()
for val in hotels['positive_review'].unique():
    arr=val.split(" ")
    for word in arr:
        word = word.strip()        
        list_tag.append(word)
        if not word in dict_tag:
            dict_tag[word] = 1
        else:            
            dict_tag[word] += 1
display(len(set(list_tag)))

# display(dict(sorted(dict_tag.items(), key=lambda item: item[1],reverse=True)))
# Закомментировал вывод слов
# Слова проанализировал для дальнейшего создания признаков

### Анализ слов в отрицательных отзывах

In [ ]:
list_tag=[]
dict_tag = dict()
for val in hotels['negative_review'].unique():
    arr=val.split(" ")
    for word in arr:
        word = word.strip()        
        list_tag.append(word)
        if not word in dict_tag:
            dict_tag[word] = 1
        else:            
            dict_tag[word] += 1
display(len(set(list_tag)))

# display(dict(sorted(dict_tag.items(), key=lambda item: item[1],reverse=True)))
# Закомментировал вывод слов
# Слова проанализировал для дальнейшего создания признаков

Выбранные слова разделил категории:
+ Наличие отзывов другого типа
+ Местоположение
+ Внутреннее убранство номера
+ Еда
+ Обслуживание
+ Степень приязни и неприязни;

Для указанных категорий проставил баллы.

In [ ]:
# Простановка признаков отрицательных (для порядка со знаком минус) и положительных рейтингов

# Простановка отрицательных баллов на основе наличия определенных слов в тексте отзыва
def func_negative_rating(x):
    f_negative_rating = 0
    str_positive_review = x['positive_review'].lower()
    str_negative_review = x['negative_review'].lower()
    
    if re.search(r'no positive+', str_positive_review):
        f_negative_rating -= 3
    if re.search(r'locat+|area+', str_negative_review):
        f_negative_rating -= 1
    if re.search(r'nois+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'smel+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'dirt+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r' +room|bed+', str_negative_review):
        f_negative_rating -= 1
    if re.search(r'bath+|shower+', str_negative_review):
        f_negative_rating -= 1
    if re.search(r'toilet+', str_negative_review):
        f_negative_rating -= 1

    if re.search(r'breakfast+|food+|diner+', str_negative_review):
        f_negative_rating -= 1
    if re.search(r'staff+|service+|personnel+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'rude+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'terribl+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'awful+', str_negative_review):
        f_negative_rating -= 2
    if re.search(r'leaving+', str_negative_review):
        f_negative_rating -= 1            
    if re.search(r'uncomf+', str_negative_review):
        f_negative_rating -= 1

    # Степень неприязни, выраженная в количестве недобрых слов
    f_negative_rating -= str_negative_review.count('bad')
    f_negative_rating -= str_negative_review.count('wors')
    f_negative_rating -= str_negative_review.count('poor')
    f_negative_rating -= str_negative_review.count('problem')
       
    return f_negative_rating

# Простановка положительных баллов на основе наличия определенных слов в тексте отзыва
def func_positive_rating(x):
    f_positive_rating = 0
    str_negative_review = x['negative_review'].lower()
    str_positive_review = x['positive_review'].lower()
        
    if re.search(r'no negative+', str_negative_review):
        f_positive_rating += 3
    if re.search(r'locat+|area+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'quiet+', str_positive_review):
        f_positive_rating += 1
    if re.search(r' +room|bed+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'bath+|shower+', str_positive_review):
        f_positive_rating += 1
    
    if re.search(r'breakfast+|food+|diner+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'staff+|service+|personnel+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'fantast+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'great+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'awesome+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'perfect+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'excellen+', str_positive_review):
        f_positive_rating += 2
    if re.search(r'comf+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'best+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'lovel+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'friend+', str_positive_review):
        f_positive_rating += 1
    if re.search(r'leaving+', str_positive_review):
        f_positive_rating += 1

    # Степень приязни, выраженная в количестве добрых слов
    f_positive_rating -= str_positive_review.count('well')
    f_positive_rating -= str_positive_review.count('good')
    
    return f_positive_rating

hotels['negative_rating'] = hotels.apply( lambda x : func_negative_rating(x), axis=1 )
hotels['positive_rating'] = hotels.apply( lambda x : func_positive_rating(x), axis=1 )

display(hotels.head(3))

### Создание признака средней оценки на основании отрицательных и положительных слов

Для дальнейшего создания признака среднего рейтинга на основании группировки по словам в отрицательных и положительных отзывах создал два новых признака **new_negative_rating** и **new_positive_rating**. Для уникальности каждой группировки несколько топорно использовал степени **2** для слов в отрицательных отзывах и степени **3** для слов в положительных отзывах. Плюс простые двузначные числа для *степеней неприязни* и *приязни* \

**Внимание!** \
Так как при таком детальном разбиении для ряда строк данных признак не устанавливается, то
для тех строк, у которых признак не установлен, снижаю детальность разбивки данных, поэтому несколько функций для группировки данных
каждая из которых сниежает детальность разбивки данных

Ниже функции с созданием временных признаков с первоначальной разбивкой данных

In [ ]:
# Простановка итогового отрицательного балла на основе наличия определенных слов в тексте отзыва
def func_negative_rating_1(x):
    f_negative_rating = 0
    str_positive_review = x['positive_review'].lower()
    str_negative_review = x['negative_review'].lower()
    
    if re.search(r'no positive+', str_positive_review):
        f_negative_rating -= 2**15
    if re.search(r'locat+|area+', str_negative_review):
        f_negative_rating -= 2**14
    if re.search(r'nois+', str_negative_review):
        f_negative_rating -= 2**13
    if re.search(r'smel+', str_negative_review):
        f_negative_rating -= 2**12
    if re.search(r'dirt+', str_negative_review):
        f_negative_rating -= 2**11
    if re.search(r' +room|bed+', str_negative_review):
        f_negative_rating -= 2**10
    if re.search(r'bath+|shower+', str_negative_review):
        f_negative_rating -= 2**9
    if re.search(r'toilet+', str_negative_review):
        f_negative_rating -= 2**8

    if re.search(r'breakfast+|food+|diner+', str_negative_review):
        f_negative_rating -= 2**7
    if re.search(r'staff+|service+|personnel+', str_negative_review):
        f_negative_rating -= 2**6
    if re.search(r'rude+', str_negative_review):
        f_negative_rating -= 2**5
    if re.search(r'terribl+', str_negative_review):
        f_negative_rating -= 2**4
    if re.search(r'awful+', str_negative_review):
        f_negative_rating -= 2**3 
    if re.search(r'leaving+', str_negative_review):
        f_negative_rating -= 2**2
    if re.search(r'uncomf+', str_negative_review):
        f_negative_rating -= 2

    # Степень неприязни, выраженная в количестве недобрых слов
    f_negative_rating -= str_negative_review.count('bad') * 11
    f_negative_rating -= str_negative_review.count('wors') * 13
    f_negative_rating -= str_negative_review.count('poor') * 17
    f_negative_rating -= str_negative_review.count('problem') * 19
    
    return f_negative_rating

# Простановка итогового положительного балла на основе наличия определенных слов в тексте отзыва
def func_positive_rating_1(x):
    f_positive_rating = 0
    str_negative_review = x['negative_review'].lower()
    str_positive_review = x['positive_review'].lower()
        
    if re.search(r'no negative+', str_negative_review):
        f_positive_rating += 3**18 
    if re.search(r'locat+|area+', str_positive_review):
        f_positive_rating += 3**17
    if re.search(r'quiet+', str_positive_review):
        f_positive_rating += 3**16
    if re.search(r'clean+', str_positive_review):
        f_positive_rating += 3**15             
    if re.search(r' +room|bed+', str_positive_review):
        f_positive_rating += 3**14
    if re.search(r'bath+|shower+', str_positive_review):
        f_positive_rating += 3**13
            
    if re.search(r'breakfast+|food+|diner+', str_positive_review):
        f_positive_rating += 3**12
    if re.search(r'staff+|service+|personnel+', str_positive_review):
        f_positive_rating += 3**11
    if re.search(r'fantast+', str_positive_review):
        f_positive_rating += 3**10
    if re.search(r'great+', str_positive_review):
        f_positive_rating += 3**9
    if re.search(r'awesome+', str_positive_review):
        f_positive_rating += 3**8
    if re.search(r'perfect+', str_positive_review):
        f_positive_rating += 3**7
    if re.search(r'excellen+', str_positive_review):
        f_positive_rating += 3**6
    if re.search(r'comf+', str_positive_review):
        f_positive_rating += 3**5
    if re.search(r'best+', str_positive_review):
        f_positive_rating += 3**4
    if re.search(r'lovel+', str_positive_review):
        f_positive_rating += 3**3
    if re.search(r'friend+', str_positive_review):
        f_positive_rating += 3**2
    if re.search(r'leaving+', str_positive_review):
        f_positive_rating += 3

    # Степень приязни, выраженная в количестве добрых слов
    f_positive_rating -= str_positive_review.count('well') * 29
    f_positive_rating -= str_positive_review.count('good') * 31
    
    return f_positive_rating

hotels['new_negative_rating_1'] = hotels.apply( lambda x : func_negative_rating_1(x), axis=1 )
hotels['new_positive_rating_1'] = hotels.apply( lambda x : func_positive_rating_1(x), axis=1 )

display(hotels.head(3))

Ниже функции с созданием временных признаков со снижением детальности разбивки данных

In [ ]:
# Простановка итогового отрицательного балла на основе наличия определенных слов в тексте отзыва
def func_negative_rating_2(x):
    f_negative_rating = 0
    str_positive_review = x['positive_review'].lower()
    str_negative_review = x['negative_review'].lower()
    
    if re.search(r'no positive+', str_positive_review):
        f_negative_rating -= 2**9
    if re.search(r'locat+|area+|nois+', str_negative_review):
        f_negative_rating -= 2**8
    if re.search(r' +room|bed+', str_negative_review):
        f_negative_rating -= 2**7
    if re.search(r'bath+|shower+|toilet+', str_negative_review):
        f_negative_rating -= 2**6

    if re.search(r'breakfast+|food+|diner+', str_negative_review):
        f_negative_rating -= 2**5
    if re.search(r'smel+|dirt+|staff+|service+|personnel+|rude+|leaving+', str_negative_review):
        f_negative_rating -= 2**4

    # Степень неприязни
    if re.search(r'terribl+|awful+', str_negative_review):
        f_negative_rating -= 2**3
    if re.search(r'bad+|wors+', str_negative_review):
        f_negative_rating -= 2**2
    if re.search(r'poor+|problem+|uncomf+', str_negative_review):
        f_negative_rating -= 2
    
    return f_negative_rating

# Простановка итогового положительного балла на основе наличия определенных слов в тексте отзыва
def func_positive_rating_2(x):
    f_positive_rating = 0
    str_negative_review = x['negative_review'].lower()
    str_positive_review = x['positive_review'].lower()
        
    if re.search(r'no negative+', str_negative_review):
        f_positive_rating += 3**9 
    if re.search(r'locat+|area+|quiet+|', str_positive_review):
        f_positive_rating += 3**8
    if re.search(r' +room|bed+', str_positive_review):
        f_positive_rating += 3**7
    if re.search(r'bath+|shower+', str_positive_review):
        f_positive_rating += 3**6
    
    if re.search(r'breakfast+|food+|diner+', str_positive_review):
        f_positive_rating += 3**5
    if re.search(r'clean+|staff+|service+|personnel+|leaving+', str_positive_review):
        f_positive_rating += 3**4

    # Степень приязни
    if re.search(r'fantast+|great+|perfect+', str_positive_review):
        f_positive_rating += 3**3
    if re.search(r'awesome+|excellen+|love+|friend+', str_positive_review):
        f_positive_rating += 3**2
    if re.search(r'best+|comf+|well+|good+', str_positive_review):
        f_positive_rating += 3
    
    return f_positive_rating

hotels['new_negative_rating_2'] = hotels.apply( lambda x : func_negative_rating_2(x), axis=1 )
hotels['new_positive_rating_2'] = hotels.apply( lambda x : func_positive_rating_2(x), axis=1 )

display(hotels.head(3))

Ниже функции с созданием временных признаков со снижением детальности разбивки данных

In [ ]:
# Простановка итогового отрицательного балла на основе наличия определенных слов в тексте отзыва
def func_negative_rating_3(x):
    f_negative_rating = 0
    str_positive_review = x['positive_review'].lower()
    str_negative_review = x['negative_review'].lower()
    
    if re.search(r'no positive+', str_positive_review):
        f_negative_rating -= 2**6
    if re.search(r'locat+|area+|nois+', str_negative_review):
        f_negative_rating -= 2**5
    if re.search(r'smel+|dirt+|staff+|service+|personnel+|rude+|leaving+', str_negative_review):
        f_negative_rating -= 2**4
    if re.search(r' +room|bed+|bath+|shower+|toilet+', str_negative_review):
        f_negative_rating -= 2**3
    if re.search(r'breakfast+|food+|diner+', str_negative_review):
        f_negative_rating -= 2**2
        
        # Степень неприязни
    if re.search(r'terribl+|awful+|bad+|wors+|poor+|problem+|uncomf+', str_negative_review):
        f_negative_rating -= 2
           
    return f_negative_rating

# Простановка итогового положительного балла на основе наличия определенных слов в тексте отзыва
def func_positive_rating_3(x):
    f_positive_rating = 0
    str_negative_review = x['negative_review'].lower()
    str_positive_review = x['positive_review'].lower()
        
    if re.search(r'no negative+', str_negative_review):
        f_positive_rating += 3**6 
    if re.search(r'locat+|area+|quiet+', str_positive_review):
        f_positive_rating += 3**5
    if re.search(r'clean+|staff+|service+|personnel+|leaving+', str_positive_review):
        f_positive_rating += 3**4
    if re.search(r' +room|bed+|bath+|shower+', str_positive_review):
        f_positive_rating += 3**3
    if re.search(r'breakfast+|food+|diner+', str_positive_review):
        f_positive_rating += 3**2

    # Степень приязни
    if re.search(r'fantast+|great+|perfect+|awesome+|excellen+|love+|friend+|best+|comf+|well+|good+', str_positive_review):
        f_positive_rating += 3
    
    return f_positive_rating

hotels['new_negative_rating_3'] = hotels.apply( lambda x : func_negative_rating_3(x), axis=1 )
hotels['new_positive_rating_3'] = hotels.apply( lambda x : func_positive_rating_3(x), axis=1 )

display(hotels.head(3))

<a id="9"></a> <br>
## Слова в Тегах

Анализ тегов в признаке **tags**

In [ ]:
# Функция для выделения чистого тега (без доп. символов)
def check_word(word):
    word = word.replace("[","")
    word = word.replace("]","")
    word = word.replace("\'","")
    word = word.strip()
    return word

tag_list=[] # Создание листа с тегами / будет использоваться в дальнейшем
dict_tag = dict() # Создание словаря для определения количества тегов в признаке
for val in hotels['tags'].unique():
    arr=val.split(",")
    for word in arr:
        word = check_word(word)
        if not word in dict_tag:
            dict_tag[word] = 1
        else:            
            dict_tag[word] += 1
display(len(set(tag_list)))

display(dict(sorted(dict_tag.items(), key=lambda item: item[1],reverse=True)))
tag_list = list(  dict(sorted(dict_tag.items(), key=lambda item: item[1],reverse=True)).keys()  )

In [ ]:
tag_list = tag_list [:1000]
display(tag_list)

In [ ]:
def func_tag_rating(x):
    f_tag_rating = 0
    count_night_tag = 0
    str_tag = x['tags'].lower()
    degree = 1
    
    for tag in tag_list:
        if tag in str_tag :
            f_tag_rating += (2**degree)
        degree += 1    

    if re.search(r'(stayed [0-9]+ night)',str_tag):    
        str_night_tag = re.search(r'(stayed [0-9]+ night)',str_tag).group(1)        
        str_count_night_tag =  re.split(r"\s+", str_night_tag)[1]      
        count_night_tag =  3 * int( str_count_night_tag ) if str_count_night_tag else 0

    f_tag_rating += count_night_tag    
    return f_tag_rating

hotels['tag_rating'] = hotels.apply( lambda x : func_tag_rating(x), axis=1 )

<a id="10"></a> <br>
## Координаты отеля

Для отелей, у которых нет координат будут проставлены координаты города и страны его нахождения

In [ ]:
# В качестве небольшого упрощения дальнейшей обработки
hotels['hotel_address'] = hotels['hotel_address'].replace("United Kingdom","UK")

In [ ]:
# Маска для строк, где нет координат
mask_lng_lat_nan = hotels['lat'].isna()

In [ ]:
display(hotels[mask_lng_lat_nan][['hotel_address', 'lat','lng']].head(5))
display(hotels[mask_lng_lat_nan].shape[0])

Для сокращения времени обработки выделил уникальные адреса отелей, у которых не заполнены координаты

In [ ]:
un_adr_latlng = hotels[mask_lng_lat_nan].sort_values('hotel_address', ascending=False).drop_duplicates(['hotel_address'])
un_adr_latlng = un_adr_latlng.reset_index(drop=True)

Опредение широты и долготы по адресу отеля (город, страна)

In [ ]:
def city_country(x):
    address_split = x['hotel_address'].strip().split(" ")
    country = address_split[-1].strip()
    if country == "UK":
        country = "United Kingdom"
    
    city = address_split[-2].strip()
    return  pd.Series([country, city])

def lat_lng(city_country):
    geocode = RateLimiter( geolocator.geocode, min_delay_seconds=1 )
    geo_loc = geocode(city_country, addressdetails=True)
    
    lat = geo_loc.latitude
    lng = geo_loc.longitude

    return pd.Series([lat, lng])

un_adr_latlng[['hotel_country', 'hotel_city']] = un_adr_latlng.apply(lambda x : city_country(x), axis=1)

un_adr_latlng[['lat_new', 'lng_new']] = un_adr_latlng.apply( lambda x : lat_lng(f"{x['hotel_city']}, {x['hotel_country']}"),axis=1 )

display(un_adr_latlng.head(5))

In [ ]:
# Возвращаем первоначальное название UK
hotels['hotel_address'] = hotels['hotel_address'].replace("UK", "United Kingdom")

Простановка найденных значений широты и долготы, для строк, у которых их не было

In [ ]:
def f_fill_lat(hotel_address):
    f_mask = (un_adr_latlng['hotel_address'] == hotel_address)
    lat = un_adr_latlng[f_mask]['lat_new'].values[0]
    return lat

def f_fill_lng(hotel_address):
    f_mask = (un_adr_latlng['hotel_address'] == hotel_address)
    lng = un_adr_latlng[f_mask]['lng_new'].values[0]
    return lng

mask = mask_lng_lat_nan 
hotels.loc[mask, 'lat'] = hotels[mask].apply(lambda x: f_fill_lat(x['hotel_address']), axis=1)
hotels.loc[mask, 'lng'] = hotels[mask].apply(lambda x: f_fill_lng(x['hotel_address']), axis=1)

In [ ]:
# Проверяем заполнения
display(hotels[mask][['hotel_address', 'lat','lng']].head(5))
display(hotels[mask].shape[0])

<a id="11"></a> <br>
## Координаты гостя отеля

В дальнейшем будет создан признак с завимостью оценки гостя от расстояния, где живет гость до отеля.
Для этого ниже определил координаты гостя (по столице страны, где он живёт

In [ ]:
# Для уверенности в сравнении перевожу данные в нижний регистр
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].str.strip().str.lower()
country_data['CountryName'] = country_data['CountryName'].str.strip().str.lower()

Широта гостя будет признак **lat_rev**, долгота гостя будет признак **lng_rev**. Эти данные есть во впомогательной таблице **country_data**

In [ ]:
# Объединяем данные
hotels = hotels.merge(
    country_data[['CountryName','CapitalLatitude','CapitalLongitude']],
    how="left",
    left_on=['reviewer_nationality'],
    right_on=['CountryName']
    )
hotels.drop(['CountryName'], axis=1, inplace=True)
hotels.rename(columns={'CapitalLatitude':'lat_rev', 'CapitalLongitude':'lng_rev'},inplace=True)

In [ ]:
display(hotels.head(3))

Для данных, где национальность гостя не оперделена, буду считать, что живёт в отеле

In [ ]:
# Выделяем данные, где национальность гостя не определена
mask_not_nation = hotels['reviewer_nationality'].str.len() < 2
display(hotels[mask_not_nation].head(3))

In [ ]:
mask = mask_not_nation
hotels.loc[mask, 'lat_rev'] = hotels[mask]['lat']
hotels.loc[mask, 'lng_rev'] = hotels[mask]['lng']

Определяю, где данные отсутствуют и заполняю их

In [ ]:
mask_not_lat_rev = hotels['lat_rev'].isna()
display(hotels[mask_not_lat_rev].tail(3))

In [ ]:
mask = mask_not_lat_rev
un_rev_nation = hotels[mask].sort_values('reviewer_nationality', ascending=True)\
    .drop_duplicates(['reviewer_nationality'])
display(un_rev_nation.head(3))
display(un_rev_nation.shape[0])
display(un_rev_nation[['reviewer_nationality','lat_rev','lng_rev']])

Попадаются забавные названия места проживания гостя, но буду искать даже по ним

In [ ]:
def rev_lat_lng(x):
    rev_country = x['reviewer_nationality']   
    geocode = RateLimiter( geolocator.geocode, min_delay_seconds=1 )
    geo_loc = geocode(rev_country, addressdetails=True)        
    lat = geo_loc.latitude
    lng = geo_loc.longitude        
        
    return pd.Series([lat, lng])

un_rev_nation[['lat_rev', 'lng_rev']] = un_rev_nation.apply( lambda x : rev_lat_lng(x), axis=1 )

display(un_rev_nation[['reviewer_nationality','lat_rev','lng_rev']])

Заполнение ранее незаполненных данных

In [ ]:
def f_fill_lat_rev(reviewer_nationality):
    f_mask = (un_rev_nation['reviewer_nationality'] == reviewer_nationality)
    lat = un_rev_nation[f_mask]['lat_rev'].values[0]
    return lat

def f_fill_lng_rev(reviewer_nationality):
    f_mask = (un_rev_nation['reviewer_nationality'] == reviewer_nationality)
    lng = un_rev_nation[f_mask]['lng_rev'].values[0]
    return lng

mask = mask_not_lat_rev
hotels.loc[mask, 'lat_rev'] = hotels[mask].apply(lambda x: f_fill_lat_rev(x['reviewer_nationality']), axis=1)
hotels.loc[mask, 'lng_rev'] = hotels[mask].apply(lambda x: f_fill_lng_rev(x['reviewer_nationality']), axis=1)

<a id="12"></a> <br>
## Прочие преобразования признаков

Решил для логики сделать признак количества отрицательных слов со знаком минус.

In [ ]:
hotels['review_total_negative_word_counts'] = hotels['review_total_negative_word_counts']*(-1)

In [ ]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

Предположил, что отзывы отелей должны зависеть от года и месяца отзыва. \ 
Это логично, так как уровень отелей и впечатление от него может меняться от месяца года. \
Для дальнейшего создания признака среднего рейтинга на основании группировки по году и месяцу создал признаки **abs_review_month** и **abs_review_quarter**

In [ ]:
hotels['review_date'] = pd.to_datetime(hotels['review_date']) # Перевод признака в формат даты

# Для уникальности признаков умножил год на 10 000 и месяц на 100 Пример дата 31.12.2020 будет переведена в число 202012
hotels['abs_review_month'] =  hotels['review_date'].dt.year * 100 + hotels['review_date'].dt.month
hotels['abs_review_quarter'] =  hotels['review_date'].dt.year * 10 + hotels['review_date'].dt.quarter

display(hotels.head(3))

<a id="13"></a> <br>
# 5. Создание признаков данных

ВНИМАНИЕ! \
Основной подход при создании признаков следующий: \
Разбиваю данные с проставленной оценкой **is_train** по группам (с высокой детализацией) и определяю для групп среднюю проставленную оценку **reviewer_score**. \
Затем эту оценку переношу на группы для **всех** данных. \
Если для каких-то данных признак не установлен, то снижаю степень детализации, то есть увеличиваю группы для простановки признака. \
И проставляю такой менее детализированный признак только для строк, у которых он не проставлен.

<a id="14"></a> <br>
## Сеть отелей

Создание нового признака **chain_average_score**, показывающего зависимость отзыва от созданных ранее признаков сети отеля **'hotel_chain** и признака **abs_review_month**.

In [ ]:
# Получение медианы для группировки данных
# перименование её и простановка нового признака через merge

hotels_group_chain = hotels[is_train].groupby(by=['hotel_chain','abs_review_month'],as_index=False)['reviewer_score'].median()
hotels_group_chain.rename(columns={'reviewer_score': 'chain_average_score'}, inplace=True)


hotels = hotels.merge(
    hotels_group_chain,
    how="left",
    on=['hotel_chain','abs_review_month']
    )
display(hotels.head(3))

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['chain_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Снижение степени детализации по периоду (квартал вместо месяца)

In [ ]:
# Получение медианы для группировки данных
# перименование её и простановка нового признака через merge
def f_hotels_group_chain_1(hotel_chain,abs_review_quarter):
    f_mask = (hotels['hotel_chain'] == hotel_chain) & (hotels['abs_review_quarter'] == abs_review_quarter) & (is_train)
    hgc = hotels.loc[f_mask]['reviewer_score'].median()
    return hgc

mask = hotels['chain_average_score'].isna()
hotels.loc[mask, 'chain_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_chain_1(x['hotel_chain'], x['abs_review_quarter']), axis=1)

# Удаление временных признаков
hotels.drop(columns=['abs_review_quarter'], inplace=True)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['chain_average_score'].isna()

display(hotels[mask].head(3))

Простановка признака для оставшихся незаполненных строк

In [ ]:
def f_hotels_group_chain_2(hotel_chain):
    f_mask = (hotels['hotel_chain'] == hotel_chain) & (is_train)
    hgc = hotels.loc[f_mask]['reviewer_score'].median()
    return hgc

mask = hotels['chain_average_score'].isna()
hotels.loc[mask, 'chain_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_chain_2(x['hotel_chain']), axis=1)

mask = hotels['chain_average_score'].isna()

display(hotels[mask])

<a id="15"></a> <br>
## Слова в отзывах

Создание нового признака **review_average_score**, показывающего зависимость отзыва от созданных ранее признаков с простановкой баллов в зависимости от положительного **new_positive_rating** и отрицательного отзыва **new_negative_rating**

In [ ]:
# Создание нового признака
hotels['plus_rating_1'] = hotels['new_negative_rating_1'] + hotels['new_positive_rating_1']

# Получение медианы для группировки данных
# перименование её и простановка нового признака через merge

hotels_group_review = hotels[is_train].groupby(by=['plus_rating_1'],as_index=False)['reviewer_score'].median()
hotels_group_review.rename(columns={'reviewer_score': 'review_average_score'}, inplace=True)


hotels = hotels.merge(
    hotels_group_review,
    how="left",
    on=['plus_rating_1']
    )
display(hotels.head(3))

# Удаление временных признаков
hotels.drop(columns={'plus_rating_1', 'new_negative_rating_1', 'new_positive_rating_1'}, inplace=True)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['review_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен. Снижение детализации.

In [ ]:
# Создание нового признака
hotels['plus_rating_2'] = hotels['new_negative_rating_2'] + hotels['new_positive_rating_2']

def f_hotels_group_review_2(plus_rating_2):
    f_mask = (hotels['plus_rating_2'] == plus_rating_2) & (is_train)
    hgr = hotels.loc[f_mask]['reviewer_score'].median()
    return hgr

mask = hotels['review_average_score'].isna()
hotels.loc[mask, 'review_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_review_2(x['plus_rating_2']), axis=1)

# Удаление временных признаков
hotels.drop(columns={'plus_rating_2', 'new_negative_rating_2', 'new_positive_rating_2'}, inplace=True)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['review_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен. Снижение детализации.

In [ ]:
# Создание нового признака
hotels['plus_rating_3'] = hotels['new_negative_rating_3'] + hotels['new_positive_rating_3']

def f_hotels_group_review_3(plus_rating_3):
    f_mask = (hotels['plus_rating_3'] == plus_rating_3) & (is_train)
    hgr = hotels.loc[f_mask]['reviewer_score'].median()
    return hgr

mask = hotels['review_average_score'].isna()
hotels.loc[mask, 'review_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_review_3(x['plus_rating_3']), axis=1)

# Удаление временных признаков
hotels.drop(columns={'plus_rating_3', 'new_negative_rating_3', 'new_positive_rating_3'}, inplace=True)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['review_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен, **median** по имени отеля.

In [ ]:
def f_hotels_group_review_hotel(hotel):
    f_mask = (hotels['hotel_name'] == hotel) & (is_train)
    hgrh = hotels.loc[f_mask]['review_average_score'].median()
    return hgrh

mask = hotels['review_average_score'].isna()
hotels.loc[mask, 'review_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_review_hotel(x['hotel_name']), axis=1)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['review_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

<a id="16"></a> <br>
## Слова в тегах

In [ ]:
hotels_group_tag = hotels[is_train].groupby(by=['tag_rating'],as_index=False)['reviewer_score'].median()
hotels_group_tag.rename(columns={'reviewer_score': 'tag_average_score'}, inplace=True)


hotels = hotels.merge(
    hotels_group_tag,
    how="left",
    on=['tag_rating']
    )

# hotels.drop(columns={'tag_rating'}, inplace=True)

display(hotels.head(3))

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['tag_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен, **median** по имени отеля

In [ ]:
def f_hotels_group_tag_hotel(hotel):
    f_mask = (hotels['hotel_name'] == hotel) & (is_train)
    hgrh = hotels.loc[f_mask]['tag_average_score'].median()
    return hgrh

mask = hotels['tag_average_score'].isna()
hotels.loc[mask, 'tag_average_score'] = hotels[mask].apply(lambda x: f_hotels_group_tag_hotel(x['hotel_name']), axis=1)

In [ ]:
mask = hotels['tag_average_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

<a id="17"></a> <br>
## Создание прочих признаков

### Признак зависимости оценки **reviewer_score** от признака **days_since_review**

In [ ]:
hotels['days_since_review_new'] = hotels.apply(lambda x : x['days_since_review'].split(" ")[0], axis=1)
hotels['days_since_review_new'] = hotels['days_since_review_new'].astype(int)

In [ ]:
print(hotels['days_since_review_new'].min(), hotels['days_since_review_new'].max())

In [ ]:
hotels['days_review_new'] = hotels['days_since_review_new'].div(70).round()

In [ ]:
hotels_days_review = hotels[is_train].groupby(by=['days_review_new'], as_index=False)['reviewer_score'].median()
hotels_days_review.rename(columns={'reviewer_score': 'days_review_score'}, inplace=True)


hotels = hotels.merge(
    hotels_days_review,
    how="left",
    on=['days_review_new']
    )
    
hotels.drop(columns=['days_review_new'], inplace=True) 
    
display(hotels.head(3)) 


### Признак зависимости оценки **reviewer_score** от введённого признака **additional_scoring_ratio**

Введение признака, показывающего какую долю от общего числа отзывов, составляют отзывы без проверки выставленных оценок.

In [ ]:
hotels['additional_scoring_ratio'] = hotels['additional_number_of_scoring'] / hotels['total_number_of_reviews']
                                                                               
min = hotels['additional_scoring_ratio'].min()
max = hotels['additional_scoring_ratio'].max()

print(min,max)

In [ ]:
hotels['add_scoring_ratio'] = (hotels['additional_scoring_ratio'] / 0.0125).round().astype(int)
print(hotels['add_scoring_ratio'].unique())

In [ ]:
hotels_add_scoring = hotels[is_train].groupby(by=['add_scoring_ratio'], as_index=False)['reviewer_score'].mean().round(1)
hotels_add_scoring.rename(columns={'reviewer_score': 'add_scoring_score'}, inplace=True)


hotels = hotels.merge(
    hotels_add_scoring,
    how="left",
    on=['add_scoring_ratio']
    )
    
hotels.drop(columns=['add_scoring_ratio'], inplace=True) 
    
display(hotels.head(3)) 

### Признак зависимости оценки **reviewer_score** от признака **total_number_of_reviews_reviewer_has_given**

In [ ]:
print(hotels['total_number_of_reviews_reviewer_has_given'].min(),hotels['total_number_of_reviews_reviewer_has_given'].max())

In [ ]:
hotels_total_rev = hotels[is_train].groupby(by=['total_number_of_reviews_reviewer_has_given'], as_index=False)['reviewer_score'].median()
hotels_total_rev.rename(columns={'reviewer_score': 'total_number_rev_score'}, inplace=True)


hotels = hotels.merge(
    hotels_total_rev,
    how="left",
    on=['total_number_of_reviews_reviewer_has_given']
    )

hotels.drop(columns=['total_number_of_reviews_reviewer_has_given'], inplace=True)    
    
display(hotels.head(3))

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['total_number_rev_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен, **median** по имени отеля.

In [ ]:
def f_hotels_group_nrev_hotel(hotel):
    f_mask = (hotels['hotel_name'] == hotel) & (is_train)
    hgrh = hotels.loc[f_mask]['total_number_rev_score'].median()
    return hgrh

mask = hotels['total_number_rev_score'].isna()
hotels.loc[mask, 'total_number_rev_score'] = hotels[mask].apply(lambda x: f_hotels_group_nrev_hotel(x['hotel_name']), axis=1)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['total_number_rev_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

### Признак зависимости оценки **reviewer_score** от признака **total_number_of_reviews**

In [ ]:
hotels_total_rev = hotels[is_train].groupby(by=['total_number_of_reviews'], as_index=False)['reviewer_score'].median()
hotels_total_rev.rename(columns={'reviewer_score' : 'total_number_score'}, inplace=True)


hotels = hotels.merge(
    hotels_total_rev,
    how="left",
    on=['total_number_of_reviews']
    )

hotels.drop(columns=['total_number_of_reviews'], inplace=True)     
    
display(hotels[~is_train].head(3))

### Признак зависимости оценки **reviewer_score** от признаков **review_total_negative_word_counts** и **review_total_positive_word_counts**

In [ ]:
print(hotels['review_total_negative_word_counts'].min(),hotels['review_total_negative_word_counts'].max())
print(hotels['review_total_positive_word_counts'].min(),hotels['review_total_positive_word_counts'].max())

In [ ]:
hotels['review_total_negative_word_for_group'] = hotels['review_total_negative_word_counts'].div(4).round()
hotels['review_total_positive_word_for_group'] = hotels['review_total_positive_word_counts'].div(4).round()

In [ ]:
hotels_total_negative_word = hotels[is_train].groupby(by=['review_total_negative_word_for_group'], as_index=False)['reviewer_score'].mean().round(1)
hotels_total_negative_word.rename(columns={'reviewer_score' : 'total_negative_word_score'}, inplace=True)


hotels = hotels.merge(
    hotels_total_negative_word,
    how="left",
    on=['review_total_negative_word_for_group']
    )

hotels.drop(columns=['review_total_negative_word_for_group'], inplace=True)   
    
display(hotels.head(3))

In [ ]:
hotels_total_positive_word = hotels[is_train].groupby(by=['review_total_positive_word_for_group'], as_index=False)['reviewer_score'].mean().round(1)
hotels_total_positive_word.rename(columns={'reviewer_score' : 'total_positive_word_score'}, inplace=True)


hotels = hotels.merge(
    hotels_total_positive_word,
    how="left",
    on=['review_total_positive_word_for_group']
    )

hotels.drop(columns=['review_total_positive_word_for_group'], inplace=True)   
    
display(hotels.head(3))

### Признак зависимости оценки **reviewer_score** от признака **reviewer_nationality**

In [ ]:
hotels_rev_nationality = hotels[is_train].groupby(by=['reviewer_nationality'],as_index=False)['reviewer_score'].median()
hotels_rev_nationality.rename(columns={'reviewer_score': 'total_reviewer_nationality_score'}, inplace=True)


hotels = hotels.merge(
    hotels_rev_nationality,
    how="left",
    on=['reviewer_nationality']
    )

    
display(hotels.head(3))


Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['total_reviewer_nationality_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен, **median** по имени отеля

In [ ]:
def f_hotels_group_nation_hotel(hotel):
    f_mask = (hotels['hotel_name'] == hotel) & (is_train)
    hgrh = hotels.loc[f_mask]['total_reviewer_nationality_score'].median()
    return hgrh

mask = hotels['total_reviewer_nationality_score'].isna()
hotels.loc[mask, 'total_reviewer_nationality_score'] = hotels[mask].apply(lambda x: f_hotels_group_nation_hotel(x['hotel_name']), axis=1)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['total_reviewer_nationality_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

### Признак зависимости оценки **reviewer_score** от создаваемого признака **globus_zone**

Несколько топорно соотнёс координаты отелей зонам на глобусе ( */ 1* один градус это около 100 км)

In [ ]:
hotels['globus_zone'] = (hotels['lat'] / 1).round(0).astype(int)* 1000 + (hotels['lng'] / 1).round(0).astype(int)

In [ ]:
mask = hotels['train'] == 1
hotels_group = hotels[mask]
hotels_group = hotels_group.reset_index(drop=True)

hotels_group_globus = hotels_group.groupby(by=['globus_zone','abs_review_month'],as_index=False)['reviewer_score'].median()
hotels_group_globus.rename(columns={'reviewer_score': 'globus_average_score'}, inplace=True)

hotels = hotels.merge(
    hotels_group_globus,
    how="left",
    on=['globus_zone','abs_review_month']
    )
display(hotels.head(3))

### Признак зависимости оценки **reviewer_score** от создаваемого признака **dist_rev_hotel**

Создание признака определяющего расстояние от отеля до страны проживания гостя

In [ ]:
def dist_hotels(x,coord_1,coord_2):
    return distance.distance(coord_1,coord_2).km

hotels['dist_rev_hotel'] = hotels.apply( lambda x : dist_hotels(x,(x['lat'],x['lng']), (x['lat_rev'],x['lng_rev'])), axis=1)

hotels = hotels.round( { 'dist_rev_hotel' : 0 } )
hotels = hotels.astype( { 'dist_rev_hotel' : 'int' } )

In [ ]:
hotels['dist_rev_hotel_scale'] = hotels['dist_rev_hotel'].div(50).round(1)

In [ ]:
mask = hotels['train'] == 1
hotels_group = hotels[mask]
hotels_group = hotels_group.reset_index(drop=True)

hotels_dist_rev = hotels_group.groupby(by=['dist_rev_hotel_scale'],as_index=False)['reviewer_score'].mean().round(1)
hotels_dist_rev.rename(columns={'reviewer_score': 'dist_rev_score'}, inplace=True)

hotels = hotels.merge(
    hotels_dist_rev,
    how="left",
    on=['dist_rev_hotel_scale']
    )

hotels.drop(columns={'dist_rev_hotel_scale'}, inplace=True)

display(hotels.head(3))

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['dist_rev_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен. Снижение детализации.

In [ ]:
hotels['dist_rev_hotel_scale_2'] = hotels['dist_rev_hotel'].div(100).round(1)

In [ ]:
def f_hotels_group_dist_2(dist_rev_sacle_2):
    f_mask = (hotels['dist_rev_hotel_scale_2'] == dist_rev_sacle_2) & (is_train)
    hgr = hotels.loc[f_mask]['dist_rev_score'].median()
    return hgr

mask = hotels['dist_rev_score'].isna()
hotels.loc[mask, 'dist_rev_score'] = hotels[mask].apply(lambda x: f_hotels_group_dist_2(x['dist_rev_hotel_scale_2']), axis=1)

hotels.drop(columns={'dist_rev_hotel_scale_2'}, inplace=True)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['dist_rev_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

Простановка признака для строк, у которых он не заполнен. Снижение детализации до далёких заморских стран.

In [ ]:
def f_hotels_group_dist_3():
    f_mask = (hotels['dist_rev_hotel'] > 6_000) & (is_train)
    hgr = hotels.loc[f_mask]['dist_rev_score'].median()
    return hgr

mask = hotels['dist_rev_score'].isna()
hotels.loc[mask, 'dist_rev_score'] = hotels[mask].apply(lambda X : f_hotels_group_dist_3(), axis=1)

Проверка для всех ли строк был проставлен признак

In [ ]:
mask = hotels['dist_rev_score'].isna()

display(hotels[mask].head(3))
display(hotels[mask].shape[0])

<a id="18"></a> <br>
# 6. Анализ созданных признаков

Общий стандартный анализ

In [ ]:
hotels.info()

Убираем признаки которые "object"

In [ ]:
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
all_columns = [s for s in hotels.columns]
display(hotels[all_columns].head(3))

hotels.drop(object_columns, axis = 1, inplace=True)

In [ ]:
hotels.info()

В ходе получения финального результата какие-то столбцы удалил

In [ ]:
hotels = hotels.drop(['lat_rev', 'lng_rev', 'lat', 'lng', 'additional_number_of_scoring',
                      'globus_zone',  'abs_review_month', 'review_date' #, 'additional_scoring_ratio'#,'dist_rev_hotel'
                      # 'review_total_negative_word_counts', 'review_total_positive_word_counts'                              
                      ], axis=1)

In [ ]:
hotels.info()

Дубликаты посмотрел, но удалил ниже для **train**

In [ ]:
mask = hotels.duplicated()
hotels_duplicates = hotels[mask]
print(f'Число найденных дубликатов: {hotels_duplicates.shape[0]}')
display(hotels[mask].head(3))

Количество уникальных значений признаков

In [ ]:
list_col = [(s,len(hotels[s].unique())) for s in hotels.columns]
display(list_col)

Cписок неинформативных признаков

In [ ]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in hotels.columns:
    #наибольшая относительная частота в признаке
    top_freq = hotels[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = hotels[col].nunique() / hotels[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Построение и вывод тепловой матрицы для анализа корреляций между признаками

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,24))

sns_data = hotels.iloc[::100, :]
sns_data = sns_data.reset_index(drop=True)

sns.heatmap(
    data = sns_data.corr(numeric_only=True), 
    annot=True, 
    fmt=".2f",
    cmap= 'coolwarm',
    linewidths=0.5, 
    linecolor='black',
    cbar_kws= {'orientation': 'horizontal'} 
    )

plt.tick_params(
     labeltop=True,
     rotation=0
     )
plt.xticks(rotation='vertical')

plt.show()



Между рядом признаков есть корреляции. Чуть менял, добавлял признаки, но в итоге смотрел на значение показателя MAPE

<a id="19"></a> <br>
# 7. Обучение модели

In [ ]:
# Теперь выделим тестовую часть
train_data = hotels.query('train == 1').drop(['train'], axis=1)
test_data = hotels.query('train == 0').drop(['train'], axis=1)

In [ ]:
# Удаляем дубликаты из данных для обучения модели:
train_data = train_data.drop_duplicates()
print('Размер таблицы после удаления дубликатов: {}'.format(train_data.shape))

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)

X = train_data.drop(['reviewer_score'], axis = 1)  
y = train_data['reviewer_score'] 
display(X.head(3))
display(y.head(3))

In [ ]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [ ]:
display(hotels.info())

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))
display(y_test.head(3))

In [ ]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [ ]:
display(pd.DataFrame(y_pred))
display(y_test)

<a id="20"></a> <br>
# 8. Проверка и выгрузка данных

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)
hotels_submission['reviewer_score'] = regr.predict(test_data)
hotels_submission.to_csv('submission.csv', index=False)

Итоговый результат плохой. Поздно увидел, что можно было сделать всё совсем по-другому. Но времени нет.